In [23]:
import requests
import time, random
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [46]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
# request_payload_file = '/data/Magicoder_payloads.txt'
request_payload_file = '/data/Promptshett_codegen_50.txt'
report_path = '/data/vLLM_50prompts.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [24]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "https://refract.fosfor.com/v1/completions"
lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
# request_payload_file = '/data/Magicoder_payloads.txt'
request_payload_file = '/data/Promptshett_codegen_50.txt'
report_path = '/data/vLLM_50prompts.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])
print(df)

Empty DataFrame
Columns: [Prompt_No, Prompt, Response, Response_status, Time (seconds)]
Index: []


In [25]:
prompt=["""You are an exceptional dbt analytics engineer writes dbt models, also creates very precise and accurate functional documentation to ensure any layman understands the code written by you.
follow instruction and response accuratly. finally only generate respose without instruction. 
 
@@Instruction
this is the sample dbt model
with customer_revenue as (
select
customer_id,
sum(amount) as total_revenue
from
{ ref('orders') }
group by 1
)
 
select
c.customer_id,
c.first_name,
c.last_name,
cr.total_revenue
from
{ source('master_data','customers') } c
left join customer_revenue cr on c.customer_id = cr.customer_id

@@response
Below is the sample dbt model documentation generated for above instruction dbt model:
 
* Overview: This dbt model calculates the total revenue per customer by leveraging the data from the orders model and customers source. 
* Dependencies:
* orders: dbt model that contains information about individual orders, including the customer_id and amount.
* customers: dbt source contains customer details, including customer_id, first_name, and last_name.
* Transformation:
* The CTE 'customer_revenue' aggregates the total revenue for each customer by summing the amount from the orders model. It utilizes the customer_id field for grouping.
* The final output includes customer details from the customers table (c) and their corresponding total revenue from the customer_revenue CTE (cr). The join is performed using the customer_id field.
* Example Usage:  The resulting model can be used for various analytical purposes, such as identifying high-value customers or generating reports on customer revenue.

@@ Instruction
create similar dbt model documentation for following dbt model code
 {% snapshot users_snapshot %}  
 
{{
  config(      
    target_schema='snapshots',      
    strategy='timestamp',      
    unique_key='id',      
    updated_at='updated_at'    
  )  
}}  
 
  select * 
  from {{ source('raw','users') }}
 
{% endsnapshot %}

@@ response""",
       "Generate a DBT model which gives me variants of the products which has highest sales",
       "Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios.",
       "Define an incremental dbt model named user_activity_daily with a unique key on date_day for daily active users. Implement the 'delete+insert' strategy for incremental updates."
       ]

In [26]:
 payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [13]:
payload = {"prompt": prompt, 
           "max_tokens": 2400,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [27]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = line.strip()
#             prompt = json.loads(line.strip())
            yield prompt
#             yield prompt['payload'

In [28]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result,a,b
    return calculate_execution_time

In [29]:
def save_report(file_path, df):
    print("Saving report\n", df)
    df.to_csv(file_path, index=False)

In [30]:
@my_decorator
def inference(prompt_index, payload_data=None):
    headers = {
        'Content-Type': 'application/json',"AUTHORIZATION": f"Bearer {lltoken}"
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
#     print(f"Prompt {prompt_index}==> {payload_data['prompt']} \n {response.text}")
    print(f"Prompt {prompt_index}==> Response \n {response.text}")
    return response, payload_data['prompt'], prompt_index
    
        

In [43]:
def my_worker(thread, payload):
    prompts = read_payloads(request_payload_file)
    try:
        for line, prompt in enumerate(prompts,start=0):
            payload = {**payload, 'prompt': prompt}
            print(f"Prompting prompt {line+1}")
            a,b,c,_ = inference(line, payload)
            print(f"Finished Prompting prompt {line+1}")
    except Exception as e:
        print(a,b,c)
        print(f"Error in thread {thread} - prompt {line}: {e}")

In [44]:
#Model Inference(Request)
# if __name__ == "__main__":
#     global df
#     with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
#         futures = [executor.submit(my_worker, _, payload) for _ in range(30)]
#         concurrent.futures.wait(futures)
#     save_report(report_path, df)

In [ ]:
# #Model Inference(Request) Same Payload
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    print(len(prompts))
    index=0
    global df
    for i in range(1000):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(inference, srno, {**payload, 'prompt': prompts[random.randint(0,49)]}) for srno in range(10)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

50
loop 0+++++++++++++++++++++++++++++
loop 1+++++++++++++++++++++++++++++
Prompt 3==> Response 
 {"id":"cmpl-e86ef2c5c5664b1a819dfbe38439b6a1","object":"text_completion","created":10371,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nAssuming you have two tables:\n\n1. Employees:\n\n| id | name | manager_id |\n|----|------|------------|\n| 1  | John | 2          |\n| 2  | Jane | 3          |\n| 3  | Bob  | 4          |\n| 4  | Alice| 5          |\n\n2. Tickets:\n\n| id | employee_id | priority |\n|----|-------------|----------|\n| 1  | 1           | high     |\n| 2  | 2           | medium   |\n| 3  | 3           | high     |\n| 4  | 4           | low      |\n| 5  | 5           | high     |\n\nThe SQL query should return the details of the manager whose employee has been assigned high priority tickets.\n\nHere is the SQL query:\n\n```sql\nSELECT e.name AS Manager_Name, e.id AS Manager_ID\n

loop 3+++++++++++++++++++++++++++++
Prompt 3==> Response 
 {"id":"cmpl-d1c13342f38a4f91996e27b6e6573e45","object":"text_completion","created":10640,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```sql\n{{ config(materialized='view') }}\n\nwith customers as (\n    select * from {{ ref('stg_customers') }}\n),\n\norders as (\n    select * from {{ ref('stg_orders') }}\n),\n\ncustomer_orders as (\n    select\n        customers.customer_id,\n        customers.first_name,\n        customers.last_name,\n        count(orders.order_id) as total_orders,\n        sum(orders.amount) as total_amount_spent\n    from customers\n    left join orders using (customer_id)\n    group by 1, 2, 3\n)\n\nselect * from customer_orders\n```\n\nThis view will be used to analyze customer engagement metrics in the future.","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":57,"total_tokens":237,"comple

loop 5+++++++++++++++++++++++++++++
Prompt 8==> Response 
 {"id":"cmpl-68e2b9470f454ff9b3973a328df636a6","object":"text_completion","created":10895,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nClient_Details Table:\n\n| client_id | client_name | budget |\n|-----------|-------------|--------|\n| 1         | Client1     | 1000   |\n| 2         | Client2     | 2000   |\n| 3         | Client3     | 3000   |\n\nI want to generate a SQL query for dbt model which gives me the client details whose budget is highest.\n\nHere is the SQL query:\n\n```sql\nSELECT *\nFROM Client_Details\nORDER BY budget DESC\nLIMIT 1;\n```\n\nThis SQL query will return the client details with the highest budget.","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":19,"total_tokens":160,"completion_tokens":141}}
Prompt 3==> Response 
 {"id":"cmpl-fc306151f13a435ab3d23613debf76cd","object":"text_completion

loop 6+++++++++++++++++++++++++++++
Prompt 5==> Response 
 {"id":"cmpl-4eb6ab8f87054ad790d730f2323fb6dc","object":"text_completion","created":11026,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nAssuming you have two tables:\n\n1. `employees` table with columns: `employee_id`, `name`, `country`, `client_id`\n2. `clients` table with columns: `client_id`, `client_name`, `country`\n\nThe SQL query would look something like this:\n\n```sql\nSELECT e.employee_id, e.name, e.country, c.client_name, c.country\nFROM employees e\nJOIN clients c ON e.client_id = c.client_id\nWHERE c.country = 'US'\n```\n\nThis SQL statement will join the `employees` and `clients` tables on the `client_id` column, and then filter the results to only include rows where the `country` of the `clients` table is 'US'. The resulting table will contain the `","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_token

loop 7+++++++++++++++++++++++++++++
Prompt 5==> Response 
 {"id":"cmpl-622ba8290e98498da64d02fef300ffbc","object":"text_completion","created":11164,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```sql\n{{ config(materialized='table') }}\n\nwith sales_data as (\n    select \n        date_trunc('month', date) as forecast_date,\n        sum(sales_amount) as total_sales\n    from {{ ref('sales') }}\n    group by 1\n)\n\nselect * from sales_data\n```\n\nIn this model, we first create a CTE (Common Table Expression) named 'sales_data' that groups the sales data by month using the 'date_trunc' function and calculates the total sales for each month. Then, we select all columns from 'sales_data' and materialize the result as a table. The 'forecast_date' is specified as the unique key for this table.","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":60,"total_tokens":237,"completi

...

Prompt 0==> Response 
 {"id":"cmpl-5cb1f50e08224bfdaba4a4a31592a1f0","object":"text_completion","created":12267,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nThis is a common requirement in many businesses. I have a table named `payments` with the following columns:\n\n- `customer_id`\n- `payment_date`\n- `amount`\n\nI want to generate a dbt model that gives me a table with the following columns:\n\n- `customer_id`\n- `payment_month` (in the format 'YYYY-MM')\n- `total_payment`\n\nI'm using dbt version 1.0.0.\n\nHere is what I've tried:\n\n```sql\n{{ config(materialized='table') }}\n\nwith payments as (\n    select * from {{ ref('stg_payments') }}\n),\n\npayment_monthly as (\n    select \n        customer_id,\n        date_trunc('month', payment_date) as payment_","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":21,"total_tokens":221,"completion_tokens":200}}
Prompt 4=

Prompt 9==> Response 
 {"id":"cmpl-f443e8fa02f542b59ef650d40e7be6be","object":"text_completion","created":12279,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```sql\n{{ config(materialized='table', unique_key='product_id') }}\n\nwith inventory_stats as (\n    select\n        product_id,\n        sum(quantity) as total_quantity,\n        min(unit_price) as min_unit_price,\n        max(unit_price) as max_unit_price,\n        avg(unit_price) as avg_unit_price\n    from\n        {{ ref('inventory') }}\n    group by\n        product_id\n)\n\nselect * from inventory_stats\n```\n\nThis dbt model first calculates the aggregated statistics for each product_id in the 'inventory' model. It then selects all the calculated statistics and materializes them as a table with a unique key specified by 'product_id'.","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":66,"total_tokens":255,"c

Prompt 2==> Response 
 {"id":"cmpl-3c027806743a47d5b9b7b721d5a6ca0f","object":"text_completion","created":12291,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nI have a table named \"employee\" with columns \"emp_id\", \"emp_name\", \"manager_id\", \"dept_id\".\n\nI want to generate a sql for dbt model which gives me count of employee under each managers from each department.\n\nI tried the following sql but it's not working:\n\n```sql\nselect manager_id, dept_id, count(emp_id) as emp_count\nfrom employee\ngroup by manager_id, dept_id\n```\n\nThis sql gives me count of employees under each manager but not for each department.\n\nHow can I modify this sql to get count of employees under each manager from each department?\n\n\nA: You can use a subquery to get the count of employees under each manager from each department. Here is an example:\n\n```sql\nselect e.manager_id, e.dept_id,","logprob

Prompt 9==> Response 
 {"id":"cmpl-3037669c38a842f5bc81879f40de2332","object":"text_completion","created":12302,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```sql\n{{ config(materialized='table') }}\n\nwith sales_data as (\n    select \n        date_trunc('month', date) as forecast_date,\n        sum(sales_amount) as total_sales\n    from {{ ref('sales') }}\n    group by 1\n)\n\nselect * from sales_data\n```\n\nIn this model, we first create a CTE (Common Table Expression) named 'sales_data' that groups the sales data by month using the 'date_trunc' function and calculates the total sales for each month. Then, we select all columns from 'sales_data' and materialize the result as a table. The 'forecast_date' is specified as the unique key for this table.","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":60,"total_tokens":237,"completion_tokens":177}}
Prompt 8==> Respons

Prompt 0==> Response 
 {"id":"cmpl-9d8a08c861af4cb381d1a26d4edae60a","object":"text_completion","created":12314,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```sql\n{{ config(\n    materialized='incremental',\n    unique_key='customer_id',\n    schema='analytics',\n    tags=[\"nightly\"]\n) }}\n\nwith source as (\n    select * from {{ source('source', 'customers') }}\n),\n\nfiltered as (\n    select * from source\n    where created_at > (\n        select max(created_at) from {{ this }}\n    )\n)\n\nselect * from filtered\n```\n\nIn this model, `{{ this }}` is a special variable that represents the current model's target table. The `max(created_at)` subquery in the `where` clause of the `filtered` CTE retrieves the maximum 'created_at' timestamp from the target table. Only records with a 'created_at' timestamp greater than this maximum timestamp are included","logprobs":null,"finish_reas